In [1]:
import numpy as np
from time import time
import matplotlib.pyplot as plt
import rustworkx as rx

from qiskit_nature.problems.second_quantization.lattice import Lattice
import sys
import os
import time

from heisenberg_model import HeisenbergModel
from qiskit_nature.mappers.second_quantization import LogarithmicMapper
from qiskit.algorithms import NumPyEigensolver
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit import Aer, transpile
from qiskit.tools.visualization import plot_histogram

import dask
from distributed import Client
import dask.array as da

from kagome_custom import custom_VQE

from qiskit.primitives import Estimator
from qiskit.algorithms.optimizers import SPSA


from scipy.optimize import differential_evolution
import toolz
from multiprocessing import Pool
from qiskit.algorithms.optimizers import SLSQP

In [2]:
if __name__ == '__main__':
    t = 1.0
    edge_list = [
        (1, 2, t),
        (2, 3, t),
        (3, 5, t),
        (5, 8, t),
        (8, 11, t),
        (11, 0, t),
        (0, 6, t),
        (6, 9, t),
        (9, 10, t),
        (10, 7, t),
        (7, 4, t),
        (4, 1, t),
        (4, 2, t),
        (2, 5, t),
        (5, 11, t),
        (11, 6, t),
        (6, 10, t),
        (10, 4, t),
    ]


    num_qubits = 12
    layers = 6
    shots = 100


    
    kagome = custom_VQE(edge_list, num_qubits, layers, shots)
    kagome.get_backend()

    

    


In [3]:
def objective(x):
    global kagome
    job = kagome.estimator.run([kagome.qc],[kagome.ham],[x])
    energy = job.result().values[0]
    #print(energy)
    return energy
    

In [8]:
def callback(xk,convergence):
    ek = objective(xk)
    #print(ek)
    file = open('remix_data1.txt','a')
    ek = objective(xk)
    file.write(str(ek))
    file.write('\n')
    file.close()


In [5]:

file = open('remix_data1.txt','w')
file.write(str(objective([0]*18)))
file.write('\n')
file.close()


In [6]:
#---------------SLSQP1-------------------------------
optimizerSLSQP = SLSQP(maxiter=500,disp=True)
initial_point0 = np.random.rand(18)*1.e-2
values0=optimizerSLSQP.minimize(fun=objective,x0=initial_point0)
file = open('remix_data1.txt','a')
file.write(str(values0.fun))
file.write('\n')
file.close()


Iteration limit reached    (Exit mode 9)
            Current function value: -3.862938525052812
            Iterations: 10
            Function evaluations: 222
            Gradient evaluations: 10


In [9]:

#-------------DE-----------------

bounds = [(-np.pi,np.pi)]*18
initial_point1 = values0.x
maxiter=25000
popsize=1
tol=1.e-5
pool = Pool(18)
parameters={'maxiter':maxiter,
           'popsize':popsize,
           'workers':pool.map,
           'polish':False,
           'strategy':'best1exp',
           'updating':'deferred'}



optimizerDE = differential_evolution(func=objective,bounds=bounds,x0=initial_point1,callback=callback,disp=True,tol=tol,**parameters)

differential_evolution step 1: f(x)= -3.86294
-3.862938525052817
differential_evolution step 2: f(x)= -3.86294
-3.862938525052817
differential_evolution step 3: f(x)= -3.86294
-3.862938525052817
differential_evolution step 4: f(x)= -3.86294
-3.862938525052817
differential_evolution step 5: f(x)= -3.86294
-3.862938525052817
differential_evolution step 6: f(x)= -3.86294
-3.862938525052817
differential_evolution step 7: f(x)= -3.86294
-3.862938525052817
differential_evolution step 8: f(x)= -3.86294
-3.862938525052817
differential_evolution step 9: f(x)= -3.86294
-3.862938525052817
differential_evolution step 10: f(x)= -3.86294
-3.862938525052817


In [10]:
#---------------SLSQP2-------------------------------
initial_point2 = optimizerDE.x
values2=optimizerSLSQP.minimize(fun=objective,x0=initial_point2)
file = open('remix_data1.txt','a')
file.write(str(values2.fun))
file.write('\n')
file.close()



Iteration limit reached    (Exit mode 9)
            Current function value: -8.560031217576913
            Iterations: 10
            Function evaluations: 214
            Gradient evaluations: 10
